# Import packages

In [155]:
#Data Manipulation
import pandas as pd
import numpy as np
import psycopg2

# Data Vizualization
import matplotlib.pyplot as plt
import seaborn as sns

#Statistics


#Machine Learning
from sklearn.metrics import roc_auc_score



# Functions

In [156]:
#load sql table
def load_sql_table(tablename):
    #select all values from table
    selectQuery = f"Select * From {tablename}"
    cur = conn.cursor()
    cur.execute(selectQuery)
    rows = cur.fetchall()
    conn.commit()

    # get column names
    cur.execute(f"Select * FROM {tablename} LIMIT 0")
    colnames = [desc[0] for desc in cur.description]

    df = pd.DataFrame(rows,columns=colnames)
    
    return df

# Read data from SQL

In [157]:
#read credential to connect to server
credentials = pd.read_json("../../credentials.json",typ = "series")
dicio = credentials.to_dict()

#connect to server
conn = psycopg2.connect(database = dicio['database'], 
                        user = dicio['user'], 
                        host= dicio['host'],
                        password = dicio['password'],
                        port = dicio['port'])

#create cursor
cur = conn.cursor()


#get all table names
cur.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
tablenames = []
for table in cur.fetchall():
    tablenames.append(table)



# Load Data

In [158]:
test_raw = load_sql_table(tablenames[0][0])
train_raw = load_sql_table(tablenames[1][0])

# 1 - Data Cleaning

In [159]:
df1 = train_raw.copy()

In [160]:
df1.dtypes

id                      object
gender                  object
age                     object
driving_license         object
region_code             object
previously_insured      object
vehicle_age             object
vehicle_damage          object
annual_premium          object
policy_sales_channel    object
vintage                 object
response                object
dtype: object

In [161]:
df1.isnull().sum()

id                      0
gender                  0
age                     0
driving_license         0
region_code             0
previously_insured      0
vehicle_age             0
vehicle_damage          0
annual_premium          0
policy_sales_channel    0
vintage                 0
response                0
dtype: int64

## 1.1 Change Variables Types

In [162]:
df1["id"] = df1["id"].astype(int)
df1["age"] = df1["age"].astype(int)


df1["annual_premium"] = df1["annual_premium"].astype(float)
df1["policy_sales_channel"] = df1["policy_sales_channel"].astype("category")
df1["vintage"] = df1["vintage"].astype(int)











# 2 - Descriptive Analysis

In [163]:
df2 = df1.copy()

In [164]:
df2.shape

(381109, 12)

In [165]:
test_raw.shape

(127037, 11)

In [166]:
numerical_variables = df2.select_dtypes(include=["int","float"])
categorical_variables = df2.select_dtypes(include=["object","category"])


## 2.1 Numerical Variables

In [167]:
describe = numerical_variables.describe().T
skew = numerical_variables.skew()
kurtosis = numerical_variables.kurtosis()

describe["skew"] = skew.round(2)
describe["kurtosis"] = kurtosis.round(2)
describe

,count,mean,std,min,25%,50%,75%,max,skew,kurtosis
id,381109.0,190555.000000,110016.836208,1.0,95278.0,190555.0,285832.0,381109.0,0.00,-1.20
age,381109.0,38.822584,15.511611,20.0,25.0,36.0,49.0,85.0,0.67,-0.57
annual_premium,381109.0,30564.389581,17213.155057,2630.0,24405.0,31669.0,39400.0,540165.0,1.77,34.00
vintage,381109.0,154.347397,83.671304,10.0,82.0,154.0,227.0,299.0,0.00,-1.20


annual_premium have high kurtosis and skew

## 2.2 Categorical Variables

In [168]:
describe = categorical_variables.describe().T
describe["%"] = describe["freq"] / describe["count"]

describe

,count,unique,top,freq,%
gender,381109,2,Male,206089,0.540761
driving_license,381109,2,1,380297,0.997869
region_code,381109,53,28.0,106415,0.279225
previously_insured,381109,2,0,206481,0.54179
vehicle_age,381109,3,1-2 Year,200316,0.525613
vehicle_damage,381109,2,Yes,192413,0.504877
policy_sales_channel,381109,155,152.0,134784,0.353663
response,381109,2,0,334399,0.877437


response, driving license and policy_sales_channel are unbalanced

In [169]:
df2["policy_sales_channel"].value_counts()/len(df2)

policy_sales_channel
152.0    0.353663
26.0     0.209127
124.0    0.194157
160.0    0.057146
156.0    0.027974
           ...   
144.0    0.000003
143.0    0.000003
84.0     0.000003
123.0    0.000003
41.0     0.000003
Name: count, Length: 155, dtype: float64

# 3 - Feature Engineering

In [170]:
df3 = df2.copy()

In [171]:
df3

,id,gender,age,driving_license,region_code,previously_insured,vehicle_age,vehicle_damage,annual_premium,policy_sales_channel,vintage,response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0
...,...,...,...,...,...,...,...,...,...,...,...,...
381104,381105,Male,74,1,26.0,1,1-2 Year,No,30170.0,26.0,88,0
381105,381106,Male,30,1,37.0,1,< 1 Year,No,40016.0,152.0,131,0
381106,381107,Male,21,1,30.0,1,< 1 Year,No,35118.0,160.0,161,0
381107,381108,Female,68,1,14.0,0,> 2 Years,Yes,44617.0,124.0,74,0


# 4 - Criação de hipóteses 